In [1]:
import numpy as np
import scipy.optimize as optimize

In [2]:
""" Optimization Algorithm """
""" New Matrix """
def newMat(x, Ut, Lt, Vt, k):
  V_new = np.zeros((Vt.shape), dtype=np.cfloat)
  if k==2:
    l0,l1 = Lt[0], Lt[1]
    V_new[0] = np.cos(x[0]) / l0
    V_new[1] = (np.sin(x[0]) / l1) * np.exp(1j*x[1])
  elif k==3:
    l0,l1,l2 = Lt[0], Lt[1], Lt[2]
    V_new[0] = np.cos(x[0]) / l0
    V_new[1] = (np.sin(x[0]) / l1) * (np.cos(x[1])) * np.exp(1j*x[2])
    V_new[2] = (np.sin(x[0]) / l2) * (np.sin(x[1])) * np.exp(1j*x[3])
  else:
    l0,l1,l2,l3 = Lt[0], Lt[1], Lt[2], Lt[3]
    V_new[0] = (np.cos(x[0]) / l0) * (np.cos(x[1]))
    V_new[1] = (np.cos(x[0]) / l1) * (np.sin(x[1])) * np.exp(1j*x[3])
    V_new[2] = (np.sin(x[0]) / l2) * (np.cos(x[2])) * np.exp(1j*x[4])
    V_new[3] = (np.sin(x[0]) / l3) * (np.sin(x[2])) * np.exp(1j*x[5])
  return V_new


""" Cost Function """
def costFn(x, Ut, Lt, Vt, A, k):
    V_new = newMat(x, Ut, Lt, Vt, k)
    Bp = np.dot(np.dot(Ut,np.diag(Lt)), V_new) 
    loss = np.linalg.norm(A - Bp*np.conjugate(Bp))
    return (loss)


In [3]:
def calcResults(k, m, n):
    print ("m = ",m,", n = ",n)
    res = np.zeros((100,2))
    for i in range(100):
        A = np.random.rand(m, n)
        A = A/A.sum(axis=0)         # Optimize column-wise
        B = np.sqrt(A)
        U, L, V = np.linalg.svd(A, full_matrices=False)
        Ut = U[:, :k]
        Vt = V[:k]
        Lt = L[:k]
        At = np.dot(np.dot(Ut,np.diag(Lt)), Vt)

        U, L, V = np.linalg.svd(B, full_matrices=False)
        Ut = U[:, :k]
        Vt = V[:k]
        Lt = L[:k]

        initial_guess = np.ones((2*n*(k-1),), dtype=np.longdouble)
        V_new = np.zeros(Vt.shape, dtype=np.cfloat)
        for col in range(Vt.shape[1]):
            result = optimize.minimize(fun=costFn, x0=initial_guess, args=(Ut,Lt,Vt[:, col],A[:,col],k),
                                    tol=1e-7, method='Nelder-Mead', options={'maxiter':1e+10})
            V_new[:,col] = newMat(result.x, Ut,Lt,Vt[:, col],k)
        res[i][0] = (np.linalg.norm(A - At))
        Bp = np.dot(np.dot(Ut,np.diag(Lt)), V_new) 
        res[i][1] = (np.linalg.norm(A - np.conjugate(Bp)*Bp))
        if i%10==0: print(i, end=' ')
    print('\n')
    return res

In [4]:
res = calcResults(k=2, m=3, n=3)

m =  3 , n =  3
0 10 20 30 40 50 60 70 80 90 



In [5]:
res

array([[0.23275754, 0.14890146],
       [0.05587526, 0.04241637],
       [0.06969944, 0.05200485],
       [0.15114031, 0.10244163],
       [0.05818908, 0.03872328],
       [0.01151359, 0.01927294],
       [0.13944358, 0.11557651],
       [0.18833081, 0.15838497],
       [0.01647215, 0.01616826],
       [0.00936046, 0.00635926],
       [0.0350174 , 0.00479874],
       [0.01741609, 0.01092263],
       [0.02967082, 0.0212926 ],
       [0.0756725 , 0.0534855 ],
       [0.02012286, 0.01264875],
       [0.18822738, 0.13243927],
       [0.06088316, 0.04140264],
       [0.09807618, 0.0751078 ],
       [0.01294334, 0.0091893 ],
       [0.09753904, 0.06982086],
       [0.07713303, 0.06097829],
       [0.11854859, 0.10308388],
       [0.07437259, 0.04941894],
       [0.18814903, 0.12112228],
       [0.28495315, 0.19092412],
       [0.13584164, 0.13149977],
       [0.20111258, 0.13626622],
       [0.06492262, 0.04102278],
       [0.17486829, 0.12282008],
       [0.00780075, 0.00085478],
       [0.

In [6]:
res.mean(axis=0)

array([0.11926839, 0.09004181])